# Марковские цепи и компания

## Задание 1

Взять два разных текста и обучить на них марковскую цепь с помощью `markovify`.

Документация: https://github.com/jsvine/markovify

In [7]:
import markovify

In [8]:
with open("./m_texts/hroniki.txt", encoding = 'utf-8') as f:
    text_hr = f.read()

In [9]:
model_hr = markovify.Text(text_hr)

In [14]:
for i in range(5):
    print(model_hr.make_sentence())

Возможно, она догадывалась, о чем Эдмунд прошеп тал Питеру в конце концов все обойдется благополучно.
- Спасибо, мисс, - сказал Питер, - ставить фонарь в таком громадном доме, и при мысли о длиннющих коридорах с рядами дверей в пустые спальни для гостей.
Сьюзен глядела на профессора во все горло, Люси вскарабкалась на Стол, чтобы схватить Аслана.
Но фавн был очень сердит на Люси за руку и сердце.
- Прекрасно, великан Рамблбаффин, - сказал Дед Мороз.


In [11]:
with open("./m_texts/stalin.txt", encoding = 'utf-8') as f:
    text_st = f.read()

In [12]:
model_st = markovify.Text(text_st)

In [15]:
for i in range(5):
    print(model_st.make_sentence())

И это означало: Великая мечта перестали существовать.
В результате арестовали почти всех действующих лиц семинарской жизни.
С фронта поступали все новые сургучные печати.
Не понимал старый большевик попросил справку о своей тайной полиции - в советской тюрьме.
Стиль, который потом принесет ему проклятый шалаш!


In [68]:
model_combo = markovify.combine([ model_hr, model_st ], [ 1, 1 ])

In [48]:
for i in range(5):
    print(model_combo.make_sentence())

По доброй воле взойдет тот, кто ни в чем не знала что, и разревелась.
- Слушаюсь и повинуюсь, о королева! - прорычал он. - Можно, я оставлю ваш платок себе на доклад горбачевских министров.
Они должны были исчезнуть...
Если и найдет, вряд ли кто сумел сдать экзамен на послушного раба.
И когда они притащили его туда, им всем спать, то есть в Нарнии своими глазами, - это страна, - сказал мистер Бобр, затем Люси, Питер и Сьюзен; замыкала шествие миссис Бобриха. - Достань лучше из ящика комода несколько чистых носовых платков...


## Задание 2

На выбор, обсудим на паре.

То, что вчера, но с русским и украинским.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os

In [3]:
ukru = 'text\tlang\n'
for path in os.listdir('ukr_corpus')[:400]:
    if os.path.isfile('./ukr_corpus/' + path):
        with open('./ukr_corpus/' + path, encoding='utf-8-sig') as f:
            text = f.read().replace('\t', ' ')
            text = text.replace('"', '')
            ukru += '"' + text + '"' + "\t" + "ukr" + "\n"
for path in os.listdir('rus_corpus')[:400]:
    if os.path.isfile('./rus_corpus/' + path):
        with open('./rus_corpus/' + path, encoding='utf-8-sig') as f:
            text = f.read().replace('\t', ' ')
            text = text.replace('"', '')
            ukru += '"' + text + '"' + "\t" + "rus" + "\n"
with open('ukru.csv', 'w', encoding='utf-8-sig') as file:
    file.write(ukru)

In [4]:
ukru = pd.read_csv('ukru.csv',sep='\t')

In [5]:
ukru.head()

,text,lang
0,"Gantz/EXA це ранобе, за мотивами манґи Gantz, ...",ukr
1,17 грудня 2004 в Японії в доповнення до 15 том...,ukr
2,У липні 2009 року Young Jump почав видавати ра...,ukr
3,"Як подяку фанатам, що зробили Gantz: Ідеальна ...",ukr
4,25 січня 2011 у світ вийшов Gantz/Origins від ...,ukr


In [6]:
trg = TfidfVectorizer(analyzer = 'char', ngram_range = (2, 2))
mat_trg = trg.fit_transform(ukru.text)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(mat_trg, ukru['lang'], random_state=42)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [9]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
y_pred = model.predict(x_test)

In [11]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        rus       0.99      1.00      1.00       102
        ukr       1.00      0.99      0.99        98

avg / total       1.00      0.99      0.99       200



In [12]:
text = 'Ще нам, браття молодії, усміхнеться доля'

In [13]:
x_new = trg.transform([text])
y_new_pred = model.predict(x_new)
y_new_pred

array(['ukr'], dtype='<U3')

In [14]:
y_new_probs = model.predict_proba(x_new)
y_new_probs

array([[0.07367015, 0.92632985]])

In [15]:
import pickle

In [26]:
with open('ukr_model','wb') as f:   #выгружаем обученный классификатор
    pickle.dump(model, f)

In [27]:
with open('trg','wb') as f:   #и векторизатор, который нужен для его работы
    pickle.dump(trg, f)